In [1]:
#default_exp contrastive_loss

In [3]:
#export
# from rsna_retro.imports import *

In [6]:
from fastai2.basics import *

In [7]:
torch.cuda.set_device(1)

## XentContrastiveLoss

In [15]:
class XentContrastiveLoss(nn.Module):
    def __init__(self, temp=0.5):
        super().__init__()
        self.temp = temp

    def forward(self, x, y):
        bs,feat = x.shape
        t2 = x.view(bs,1,feat)
        
        csim = F.cosine_similarity(x, t2, dim=-1)/self.temp
        csim_noself = csim[~torch.eye(bs).bool()].reshape(bs, bs-1)
        return F.cross_entropy(csim_noself, y)
    

#export
from pytorch_metric_learning import losses
class XentLoss(losses.NTXentLoss):
    def forward(self, output1, output2):
        stacked = torch.cat((output1, output2), dim=0)
        labels = torch.arange(output1.shape[0]).repeat(2)
        return super().forward(stacked, labels, None)
    

In [24]:
class XentContrastiveLoss2(nn.Module):
    def __init__(self, temp=0.5):
        super().__init__()
        self.temp = temp

    def forward(self, output1, output2):
        x = torch.cat((output1, output2), dim=0)
        bs = x.shape[0]
        y = (torch.arange(bs, device=x.device)+bs//2-1) % (bs-1)
        
        bs,feat = x.shape
        t2 = x.view(bs,1,feat)
        
        csim = F.cosine_similarity(x, t2, dim=-1)/self.temp
        csim_noself = csim[~torch.eye(bs).bool()].reshape(bs, bs-1)
        return F.cross_entropy(csim_noself, y)

In [25]:

#         xb, = self.learn.xb
#         xb_targ = self.aug_targ(xb)
#         xb_pos = self.aug_pos(xb)
#         self.learn.xb = torch.cat((xb_targ, xb_pos), dim=0),
#         bs = self.learn.xb[0].shape[0]
#         self.learn.yb = (torch.arange(bs, device=xb.device)+bs//2-1) % (bs-1),

In [26]:
bs,feat=4,16 # 
targ = torch.randn(bs, feat) # bs x features
rand_aug = torch.randn(bs, feat) # bs x features
aug = targ + 0.1 # bs x feat

In [32]:
o1, o2 = targ, rand_aug
temp = 1.0

In [33]:

xcl2 = XentContrastiveLoss2(temp)
xl2 = XentLoss(temp)

In [34]:
xcl2(o1, o2), xl2(o1,o2)

(tensor(2.0646), tensor(2.0646))

In [35]:
bs = 8

In [50]:
t = torch.arange(8).roll(4)
t[t>=4] -= 1

In [59]:
bs2 = bs-1; bs2

7

In [76]:
torch.eye(bs).roll(bs//2,1)[~torch.eye(bs).bool()].reshape(bs, bs-1).nonzero()

tensor([[0, 3],
        [1, 4],
        [2, 5],
        [3, 6],
        [4, 0],
        [5, 1],
        [6, 2],
        [7, 3]])

In [60]:
bs2//2

3

In [68]:
torch.cat([torch.arange(bs2//2,bs2), torch.arange(0,round(bs/2))])

tensor([3, 4, 5, 6, 0, 1, 2, 3])

In [39]:
torch.arange(4).repeat(2)

tensor([0, 1, 2, 3, 0, 1, 2, 3])

In [37]:
y = (torch.arange(bs)+bs//2-1) % (bs-1)
y

tensor([3, 4, 5, 6, 0, 1, 2, 3])

## Without Xent

In [38]:
cos_sim

tensor([[ 0.9973, -0.5696,  0.2862, -0.1613],
        [-0.5812,  0.9949, -0.1916,  0.0144],
        [ 0.2550, -0.2069,  0.9969,  0.2842],
        [-0.1879,  0.0139,  0.3300,  0.9865]])

In [39]:
cexp = torch.exp(cos_sim)
cexp

tensor([[2.7110, 0.5657, 1.3314, 0.8510],
        [0.5592, 2.7045, 0.8256, 1.0145],
        [1.2905, 0.8131, 2.7098, 1.3287],
        [0.8287, 1.0140, 1.3910, 2.6817]])

In [40]:
cexp.sum(dim=-1)

tensor([5.4592, 5.1039, 6.1420, 5.9154])

In [41]:
neg_denom = (cexp*(1-labels)).sum(dim=-1); neg_denom

tensor([2.7481, 2.3994, 3.4322, 3.2337])

In [42]:
pos = cos_sim[range(lmax.shape[0]),lmax]; pos

tensor([0.9973, 0.9949, 0.9969, 0.9865])

In [43]:
cexp.sum(dim=-1) - torch.exp(pos)

tensor([2.7481, 2.3994, 3.4322, 3.2337])

In [44]:
lsoft2 = torch.log(cexp/neg_denom); lsoft2

tensor([[-0.0136, -1.4448, -0.9470, -1.3350],
        [-1.5921,  0.1197, -1.4248, -1.1592],
        [-0.7559, -1.0821, -0.2363, -0.8895],
        [-1.1988, -0.8613, -0.9032, -0.1872]])

In [45]:
l_nll = torch.mean(torch.sum(-lmax * lsoft2, dim=1)); l_nll

tensor(5.2509)

## Test

In [46]:
temp = 1.0
cos_sim = F.cosine_similarity(output1, output2, dim=-1)/temp
cexp = torch.exp(cos_sim)
cexp

tensor([[2.7110, 0.5657, 1.3314, 0.8510],
        [0.5592, 2.7045, 0.8256, 1.0145],
        [1.2905, 0.8131, 2.7098, 1.3287],
        [0.8287, 1.0140, 1.3910, 2.6817]])

In [47]:
x = (cexp * labels).sum(dim=-1); x

tensor([2.7110, 2.7045, 2.7098, 2.6817])

In [48]:
(cexp*(1-labels)).sum(dim=-1)

tensor([2.7481, 2.3994, 3.4322, 3.2337])

In [49]:
denom = cexp.sum(dim=-1) - x; denom

tensor([2.7481, 2.3994, 3.4322, 3.2337])

In [50]:
lsoft = -torch.log(cexp/denom)
lsoft

tensor([[ 0.0136,  1.4448,  0.9470,  1.3350],
        [ 1.5921, -0.1197,  1.4248,  1.1592],
        [ 0.7559,  1.0821,  0.2363,  0.8895],
        [ 1.1988,  0.8613,  0.9032,  0.1872]])

In [51]:
-torch.log(x)+torch.log(denom)

tensor([ 0.0136, -0.1197,  0.2363,  0.1872])

In [52]:
neg_denom = (cexp*(1-labels)).sum(dim=-1); neg_denom
lsoft1 = torch.log(cexp/neg_denom)
lsoft2 = torch.sum(-labels * lsoft1, dim=-1)
lsoft2

tensor([ 0.0136, -0.1197,  0.2363,  0.1872])

## Metric Learning

In [53]:

def sim_mat(x, y=None):
    """
    returns a matrix where entry (i,j) is the dot product of x[i] and x[j]
    """
    if y is None:
        y = x
    return torch.matmul(x, y.t())

def convert_to_pairs(indices_tuple, labels):
    """
    This returns anchor-positive and anchor-negative indices,
    regardless of what the input indices_tuple is
    Args:
        indices_tuple: tuple of tensors. Each tensor is 1d and specifies indices
                        within a batch
        labels: a tensor which has the label for each element in a batch
    """
    if indices_tuple is None:
        return get_all_pairs_indices(labels)
    elif len(indices_tuple) == 4:
        return indices_tuple
    else:
        a, p, n = indices_tuple
        return a, p, a, n

def get_all_pairs_indices(labels, ref_labels=None):
    """
    Given a tensor of labels, this will return 4 tensors.
    The first 2 tensors are the indices which form all positive pairs
    The second 2 tensors are the indices which form all negative pairs
    """
    if ref_labels is None:
        ref_labels = labels
    labels1 = labels.unsqueeze(1)
    labels2 = ref_labels.unsqueeze(0)
    matches = (labels1 == labels2).byte()
    diffs = matches ^ 1
    if ref_labels is labels:
        matches -= torch.eye(matches.size(0)).byte().to(labels.device)
    a1_idx = matches.nonzero()[:, 0].flatten()
    p_idx = matches.nonzero()[:, 1].flatten()
    a2_idx = diffs.nonzero()[:, 0].flatten()
    n_idx = diffs.nonzero()[:, 1].flatten()
    return a1_idx, p_idx, a2_idx, n_idx



In [54]:
class NTXentLoss2(nn.Module):

    def __init__(self, temperature, **kwargs):
        super().__init__(**kwargs)
        self.temperature = temperature
        self.normalize_embeddings = True

    def forward(self, embeddings, labels, indices_tuple):
        cosine_similarity = sim_mat(embeddings)
        if not self.normalize_embeddings:
            embedding_norms_mat = self.embedding_norms.unsqueeze(0)*self.embedding_norms.unsqueeze(1)
            cosine_similarity = cosine_similarity / (embedding_norms_mat)
        cosine_similarity = cosine_similarity / self.temperature

        a1, p, a2, n = convert_to_pairs(indices_tuple, labels)

        if len(a1) > 0 and len(a2) > 0:
            pos_pairs = cosine_similarity[a1, p].unsqueeze(1)
            neg_pairs = cosine_similarity[a2, n]
            n_per_p = (a2.unsqueeze(0) == a1.unsqueeze(1)).float()
            neg_pairs = neg_pairs*n_per_p
            neg_pairs[n_per_p==0] = float('-inf')

            max_val = torch.max(pos_pairs, torch.max(neg_pairs, dim=1, keepdim=True)[0])
            numerator = torch.exp(pos_pairs - max_val).squeeze(1)
            denominator = torch.sum(torch.exp(neg_pairs - max_val), dim=1) + numerator
            log_exp = torch.log((numerator/denominator) + 1e-20)
            return torch.mean(-log_exp)
        return 0

In [55]:
ml = losses.NTXentLoss(1.0)

AttributeError: 'list' object has no attribute 'NTXentLoss'

In [56]:
stacked = torch.cat((targ, rand_aug), dim=0)

In [57]:
labels = torch.arange(targ.shape[0]).repeat(2)

In [58]:
get_all_pairs_indices(labels)

(tensor([0, 1, 2, 3, 4, 5, 6, 7]),
 tensor([4, 5, 6, 7, 0, 1, 2, 3]),
 tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
         4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7]),
 tensor([1, 2, 3, 5, 6, 7, 0, 2, 3, 4, 6, 7, 0, 1, 3, 4, 5, 7, 0, 1, 2, 4, 5, 6,
         1, 2, 3, 5, 6, 7, 0, 2, 3, 4, 6, 7, 0, 1, 3, 4, 5, 7, 0, 1, 2, 4, 5, 6]))

In [3]:
#export
from pytorch_metric_learning import losses
class XentLoss(losses.NTXentLoss):
    def forward(self, output1, output2):
        stacked = torch.cat((output1, output2), dim=0)
        labels = torch.arange(output1.shape[0]).repeat(2)
        return super().forward(stacked, labels, None)

In [77]:
ml(stacked, labels, None)

tensor(1.8078)

## Put it into code

In [61]:
#export
# https://arxiv.org/pdf/2002.05709.pdf
class XentContrastiveLoss(nn.Module):
    def __init__(self, temp=0.5):
        super().__init__()
        self.temp = temp

    def forward(self, output1, output2, labels):
        cos_sim = F.cosine_similarity(output1, output2, dim=-1)/self.temp
        cexp = torch.exp(cos_sim)
        neg_denom = (cexp*(1-labels)).sum(dim=-1)
        lsoft = torch.log(cexp/neg_denom)
        lsoft = torch.sum(-labels * lsoft, dim=-1)
        print(lsoft)
        return lsoft.mean()

In [201]:
#export
class XentContrastiveLoss2(nn.Module):
    def __init__(self, temp=0.5):
        super().__init__()
        self.temp = temp

    def forward(self, output1, output2, labels):
        cos_sim = F.cosine_similarity(output1, output2, dim=-1)/self.temp
        cexp = torch.exp(cos_sim)
        x = (cexp * labels).sum(dim=-1)
        denom = cexp.sum(dim=-1) - x
        lsoft = -torch.log(x/denom)
        print(lsoft)
        return lsoft.mean()

## Batch

In [63]:
#export
class BatchContrastiveLoss(nn.Module):
    def __init__(self, loss_func):
        super().__init__()
        self.loss_func = loss_func
        self.onehot = not isinstance(loss_func, XentOldContrastiveLoss)
        
    def forward(self, output1, output2):
        output1, output2, labels = batched_labels(output1, output2, self.onehot)
        return self.loss_func(output1, output2, labels)

In [78]:
temp = 0.1
ls2 = {
#     '1_contloss': ContrastiveLoss(margin=1.0),
    '4_xentloss': XentContrastiveLoss(temp=temp),
    '5_xentloss2': XentContrastiveLoss2(temp=temp),
    '6_oldxentloss': XentOldContrastiveLoss(temp=temp)
}

def batch_loss_out(t,a):
    return {k:BatchContrastiveLoss(v)(t,a) for k,v in ls2.items()}

In [79]:
bs,feat=16,128 # 
targ = torch.randn(bs, feat) # bs x features
rand_aug = torch.randn(bs, feat) # bs x features
aug = targ + 0.05 # bs x feat

In [98]:
batch_loss_out(targ,targ)

{'4_xentloss': tensor(-6.8875),
 '5_xentloss2': tensor(-6.8876),
 '6_oldxentloss': tensor(0.0010)}

In [81]:
batch_loss_out(targ,rand_aug)

{'4_xentloss': tensor(3.4285),
 '5_xentloss2': tensor(3.4285),
 '6_oldxentloss': tensor(3.4732)}

In [77]:
bs,feat=256,256 # 
targ = torch.randn(bs, feat) # bs x features
aug = targ+0.1 # bs x features
rand_aug = torch.randn(bs, feat) # bs x features
batch_loss_out(targ,-aug)

{'4_xentloss': tensor(7.5412),
 '5_xentloss2': tensor(7.5412),
 '6_oldxentloss': tensor(7.5418)}

## Export

In [4]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_metadata.ipynb.
Converted 01_preprocess.ipynb.
Converted 01_preprocess_mean_std.ipynb.
Converted 02_train.ipynb.
Converted 02_train_01_save_features.ipynb.
Converted 03_train3d.ipynb.
Converted 04_trainfull3d_deprecated.ipynb.
Converted 04_trainfull3d_labels.ipynb.
Converted 05_train_adjacent.ipynb.
Converted 06_seutao_features.ipynb.
Converted 07_adni.ipynb.
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
#
This cell doesn't have an export destination and was ignored:
e
Converted 07_adni_01.ipynb.
Converted 08_contrastive_loss-Copy1.ipynb.
Converted 08_contrastive_loss.ipynb.
Converted 08_imagewang.ipynb.
Converted 08_train_self_supervised.ipynb.
Converted 08_train_self_supervised_train_1.ipynb.
Converted 08_train_self_supervised_train_2_nocombined.ipynb.
Converted 08_train_self_supervised_train_2_nocombined_contrast.ipynb.
Converted 08_train_self_supervised_train_3.ipynb.
Converted 08_train_sel